In [3]:
import csv
import pandas as pd
import yfinance as yf
from concurrent.futures import ThreadPoolExecutor
from stocksymbol import StockSymbol
from datetime import datetime, timedelta
import pytz

# Function to retrieve symbols from the IN market and save them to a CSV file
def save_symbols_to_csv(file_path):
    api_key = '74e6f192-55d1-4115-a88b-7e8c17e29b9c'
    ss = StockSymbol(api_key)
    symbol_list_us = ss.get_symbol_list(market="IN")
    with open(file_path, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Symbol"])
        writer.writerows(zip(symbol_list_us))
    print("List of symbols (IN market) saved to symbol_list.csv file.")

# Function to clean the symbol list CSV file
def clean_symbol_list(input_file, output_file):
    df = pd.read_csv(input_file)
    df['symbol'] = df['Symbol'].apply(lambda x: eval(x)['symbol'])
    df.drop(columns=['Symbol'], inplace=True)
    df.to_csv(output_file, index=False)
    print("Symbols extracted and saved to", output_file)

# Function to fetch symbol information and save to a CSV file
def fetch_symbol_info(input_file, output_file):
    symbols_df = pd.read_csv(input_file)
    desired_features = ['sector']

    # Function to fetch symbol information
    def fetch_symbol_info(symbol):
        try:
            ticker = yf.Ticker(symbol)
            info = ticker.info
            selected_info = {'symbol': symbol}
            for feature in desired_features:
                if feature in info:
                    selected_info[feature] = info[feature]
            return selected_info
        except Exception as e:
            print(f"Error processing symbol {symbol}: {e}")
            return None

    # Concurrently fetch symbol information
    with ThreadPoolExecutor(max_workers=10) as executor:
        selected_info_list = list(executor.map(fetch_symbol_info, symbols_df['symbol']))

    selected_info_df = pd.DataFrame(selected_info_list)
    selected_info_df.to_csv(output_file, index=False)
    print("Symbol information extracted and saved to", output_file)

# Function to process sector-symbols associations and save to a CSV file
def process_sector_symbols(input_file, output_file):
    data_df = pd.read_csv(input_file)
    sector_symbols = {}
    for index, row in data_df.iterrows():
        symbol = row['symbol']
        sector = row['sector']
        sector_symbols[sector] = sector_symbols.get(sector, []) + [symbol]
    sector_symbols_df = pd.DataFrame(sector_symbols.items(), columns=['Sector', 'Symbols'])
    sector_symbols_df.to_csv(output_file, index=False)
    print("Sector-symbols associations saved to", output_file)

# Function to modify the symbols in the sector-symbols associations CSV file
def modify_symbols_in_sector_symbols(input_file, output_file):
    data_df = pd.read_csv(input_file)
    if 'Symbols' in data_df.columns:
        data_df['Symbols'] = data_df['Symbols'].apply(lambda x: ', '.join(eval(x)))
        data_df.to_csv(output_file, index=False)
        print("Symbols modified and saved to", output_file)
    else:
        print("Error: 'Symbols' column not found in the DataFrame.")

# Main script
try:
    # Define file paths
    symbol_list_file = "/content/symbol_list.csv"
    cleaned_symbol_list_file = "/content/ticker_cleaned.csv"
    symbol_info_file = "/content/symbol_info.csv"
    sector_symbols_file = "/content/sector_symbols.csv"
    modified_sector_symbols_file = "/content/sector_symbols_modified.csv"
    output_csv_file = "/content/sector_symbol_results.csv"

    # Step 1: Save symbols to CSV
    save_symbols_to_csv(symbol_list_file)

    # Step 2: Clean symbol list
    clean_symbol_list(symbol_list_file, cleaned_symbol_list_file)

    # Step 3: Fetch symbol information and save to CSV
    fetch_symbol_info(cleaned_symbol_list_file, symbol_info_file)

    # Step 4: Process sector-symbols associations and save to CSV
    process_sector_symbols(symbol_info_file, sector_symbols_file)

    # Step 5: Modify symbols in sector-symbols associations
    modify_symbols_in_sector_symbols(sector_symbols_file, modified_sector_symbols_file)

    # Read symbols from the modified sector_symbols file
    symbols_df = pd.read_csv(modified_sector_symbols_file)

    # Print available sectors and ask user for selection
    print("Select a sector for which you want to retrieve average gain and loss:")
    for index, sector in enumerate(symbols_df['Sector'], start=1):
        print(f"{index}. {sector}")

    selected_sector_index = int(input("Enter the sector number: ")) - 1
    selected_sector = symbols_df['Sector'][selected_sector_index]
    selected_symbols = symbols_df['Symbols'][selected_sector_index].split(', ')

    # Construct result list using list comprehension
    result_list = [
        (selected_sector, symbol, average_gain_1_7_days, average_loss_8_14_days)
        for symbol in selected_symbols
    ]

    # Create DataFrame from the result list
    result_df = pd.DataFrame(result_list, columns=['Sector', 'Symbol', 'Average Gain', 'Average Loss'])

    # Save the DataFrame to CSV
    result_df.to_csv(output_csv_file, index=False)

    print("\nResults have been saved to", output_csv_file)

except Exception as e:
    print("Error occurred:", e)


List of symbols (IN market) saved to symbol_list.csv file.
Symbols extracted and saved to /content/ticker_cleaned.csv


ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ADANITRANS.NS?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ADANITRANS.NS&crumb=oYMiUYeJR71
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/LTI.BO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=LTI.BO&crumb=oYMiUYeJR71
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MOTHERSUMI.BO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MOTHERSUMI.BO&crumb=oYMiUYeJR71
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ADANITRANS.BO?mo

Symbol information extracted and saved to /content/symbol_info.csv
Sector-symbols associations saved to /content/sector_symbols.csv
Symbols modified and saved to /content/sector_symbols_modified.csv
Select a sector for which you want to retrieve average gain and loss:
1. Energy
2. Technology
3. Financial Services
4. Consumer Defensive
5. nan
6. Communication Services
7. Basic Materials
8. Industrials
9. Consumer Cyclical
10. Utilities
11. Healthcare
12. Real Estate
Enter the sector number: 1
Error occurred: name 'average_gain_1_7_days' is not defined


In [9]:
import csv
import pandas as pd
import yfinance as yf
from concurrent.futures import ThreadPoolExecutor
from stocksymbol import StockSymbol
from datetime import datetime, timedelta
import pytz

# Function to retrieve symbols from the IN market and save them to a CSV file
def save_symbols_to_csv(file_path):
    api_key = '74e6f192-55d1-4115-a88b-7e8c17e29b9c'
    ss = StockSymbol(api_key)
    symbol_list_us = ss.get_symbol_list(market="IN")
    with open(file_path, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Symbol"])
        writer.writerows(zip(symbol_list_us))
    print("List of symbols (IN market) saved to symbol_list.csv file.")

# Function to clean the symbol list CSV file
def clean_symbol_list(input_file, output_file):
    df = pd.read_csv(input_file)
    df['symbol'] = df['Symbol'].apply(lambda x: eval(x)['symbol'])
    df.drop(columns=['Symbol'], inplace=True)
    df.to_csv(output_file, index=False)
    print("Symbols extracted and saved to", output_file)

# Function to fetch symbol information and save to a CSV file
def fetch_symbol_info(input_file, output_file):
    symbols_df = pd.read_csv(input_file)
    desired_features = ['sector']

    # Function to fetch symbol information
    def fetch_symbol_info(symbol):
        try:
            ticker = yf.Ticker(symbol)
            info = ticker.info
            if info is not None:  # Add a check here
                selected_info = {'symbol': symbol}
                for feature in desired_features:
                    if feature in info:
                        selected_info[feature] = info[feature]
                return selected_info
            else:
                print(f"No information found for symbol {symbol}")
                return None
        except Exception as e:
            print(f"Error processing symbol {symbol}: {e}")
            return None

    # Concurrently fetch symbol information
    with ThreadPoolExecutor(max_workers=10) as executor:
        selected_info_list = list(executor.map(fetch_symbol_info, symbols_df['symbol']))

    selected_info_df = pd.DataFrame(selected_info_list)
    selected_info_df.to_csv(output_file, index=False)
    print("Symbol information extracted and saved to", output_file)

# Function to process sector-symbols associations and save to a CSV file
def process_sector_symbols(input_file, output_file):
    data_df = pd.read_csv(input_file)
    sector_symbols = {}
    for index, row in data_df.iterrows():
        symbol = row['symbol']
        sector = row['sector']
        sector_symbols[sector] = sector_symbols.get(sector, []) + [symbol]
    sector_symbols_df = pd.DataFrame(sector_symbols.items(), columns=['Sector', 'Symbols'])
    sector_symbols_df.to_csv(output_file, index=False)
    print("Sector-symbols associations saved to", output_file)

# Function to modify the symbols in the sector-symbols associations CSV file
def modify_symbols_in_sector_symbols(input_file, output_file):
    data_df = pd.read_csv(input_file)
    if 'Symbols' in data_df.columns:
        data_df['Symbols'] = data_df['Symbols'].apply(lambda x: ', '.join(eval(x)))
        data_df.to_csv(output_file, index=False)
        print("Symbols modified and saved to", output_file)
    else:
        print("Error: 'Symbols' column not found in the DataFrame.")

# Main script
try:
    # Define file paths
    symbol_list_file = "/content/symbol_list.csv"
    cleaned_symbol_list_file = "/content/ticker_cleaned.csv"
    symbol_info_file = "/content/symbol_info.csv"
    sector_symbols_file = "/content/sector_symbols.csv"
    modified_sector_symbols_file = "/content/sector_symbols_modified.csv"
    output_csv_file = "/content/sector_symbol_results.csv"

    # Step 1: Save symbols to CSV
    save_symbols_to_csv(symbol_list_file)

    # Step 2: Clean symbol list
    clean_symbol_list(symbol_list_file, cleaned_symbol_list_file)

    # Step 3: Fetch symbol information and save to CSV
    fetch_symbol_info(cleaned_symbol_list_file, symbol_info_file)

    # Step 4: Process sector-symbols associations and save to CSV
    process_sector_symbols(symbol_info_file, sector_symbols_file)

    # Step 5: Modify symbols in sector-symbols associations
    modify_symbols_in_sector_symbols(sector_symbols_file, modified_sector_symbols_file)

    # Read symbols from the modified sector_symbols file
    symbols_df = pd.read_csv(modified_sector_symbols_file)

    # Print available sectors and ask user for selection
    print("Select a sector for which you want to retrieve average gain and loss:")
    for index, sector in enumerate(symbols_df['Sector'], start=1):
        print(f"{index}. {sector}")

    selected_sector_index = int(input("Enter the sector number: ")) - 1
    selected_sector = symbols_df['Sector'][selected_sector_index]
    selected_symbols = symbols_df['Symbols'][selected_sector_index].split(', ')

    result_list = []

    # Iterate over symbols in the selected sector
    for symbol in selected_symbols:
        # Initialize a list to store trading days
        trading_days = []

        # Get today's date
        local_tz = pytz.timezone('Asia/Kolkata')
        end_date = datetime.now(local_tz)

        # Variable to keep track of the number of trading days found
        trading_days_found = 0

        # Loop to find the last 14 trading days excluding weekends
        while trading_days_found < 19:
            # Check if the current day is a weekday (0 to 4 are Monday to Friday)
            if end_date.weekday() < 6:
                trading_days.append(end_date)
                trading_days_found += 1
            end_date -= timedelta(days=1)  # Move to the previous day

        # Reverse the list to have the dates in ascending order
        trading_days.reverse()

        # Convert dates to string format
        start_date_str = trading_days[0].strftime('%Y-%m-%d')
        end_date_str = trading_days[-1].strftime('%Y-%m-%d')

        # Fetch historical data from Yahoo Finance for the current symbol
        data = yf.download(symbol, start=start_date_str, end=end_date_str)

        # Calculate daily gain/loss percentage
        data['Daily Gain/Loss %'] = (data['Close'] - data['Open']) / data['Open'] * 100

        # Extract days 1-7 and 8-14
        days_1_to_7 = data.iloc[:7]['Daily Gain/Loss %']
        days_8_to_14 = data.iloc[7:14]['Daily Gain/Loss %']

        # Calculate the average gain for days 1-7
        average_gain_1_7_days = days_1_to_7[days_1_to_7 > 0].sum() / len(days_1_to_7[days_1_to_7 > 0]) if len(days_1_to_7[days_1_to_7 > 0]) > 0 else 0

        # Calculate the average loss for days 8-14
        average_loss_8_14_days = days_8_to_14[days_8_to_14 < 0].sum() / len(days_8_to_14[days_8_to_14 < 0]) if len(days_8_to_14[days_8_to_14 < 0]) > 0 else 0

        result_list.append([selected_sector, symbol, average_gain_1_7_days, average_loss_8_14_days])

    # Create DataFrame from the result list
    result_df = pd.DataFrame(result_list, columns=['Sector', 'Symbol', 'Average Gain', 'Average Loss'])

    # Save the DataFrame to CSV
    result_df.to_csv(output_csv_file, index=False)

    print("\nResults have been saved to", output_csv_file)

except Exception as e:
    print("Error occurred:", e)


List of symbols (IN market) saved to symbol_list.csv file.
Symbols extracted and saved to /content/ticker_cleaned.csv


ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ADANITRANS.NS?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ADANITRANS.NS&crumb=oYMiUYeJR71
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/LTI.BO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=LTI.BO&crumb=oYMiUYeJR71
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MOTHERSUMI.BO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MOTHERSUMI.BO&crumb=oYMiUYeJR71
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ADANITRANS.BO?mo

Symbol information extracted and saved to /content/symbol_info.csv
Sector-symbols associations saved to /content/sector_symbols.csv
Symbols modified and saved to /content/sector_symbols_modified.csv
Select a sector for which you want to retrieve average gain and loss:
1. Energy
2. Technology
3. Financial Services
4. Consumer Defensive
5. nan
6. Communication Services
7. Basic Materials
8. Industrials
9. Consumer Cyclical
10. Utilities
11. Healthcare
12. Real Estate
Enter the sector number: 1


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******


Results have been saved to /content/sector_symbol_results.csv


In [10]:
import pandas as pd

# Load the CSV file
input_csv_file = "/content/sector_symbol_results.csv"
output_csv_file = "output_positive_avg_loss.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(input_csv_file)

# Make entries of "avg loss" column positive
df['Average Loss'] = df['Average Loss'].abs()

# Save the modified DataFrame to a new CSV file
df.to_csv(output_csv_file, index=False)

print("Average loss entries have been made positive and saved to", output_csv_file)


Average loss entries have been made positive and saved to output_positive_avg_loss.csv


In [11]:
import pandas as pd

# Load the CSV file containing sector, symbol, average gain, and average loss
input_csv_file = "/content/output_positive_avg_loss.csv"
output_csv_file = "output_with_rsi.csv"

# Read the CSV file
df = pd.read_csv(input_csv_file)

# Calculate RSI using the formula
rsi_values = 100 - (100 / (1 + (df['Average Gain'] / df['Average Loss'])))

# Add the RSI values to the DataFrame
df['RSI'] = rsi_values

# Save the DataFrame with RSI values to a new CSV file
df.to_csv(output_csv_file, index=False)

print("RSI values have been calculated and saved to", output_csv_file)


RSI values have been calculated and saved to output_with_rsi.csv


In [12]:
import pandas as pd

# Read the CSV file
file_path = '/content/output_with_rsi.csv'  # Specify the path to your CSV file
data = pd.read_csv(file_path)

# Drop the specified columns
columns_to_drop = ['Sector', 'Average Gain', 'Average Loss']
data.drop(columns=columns_to_drop, inplace=True)

# Save the modified DataFrame to a new CSV file
new_file_path = 'new_file.csv'  # Specify the path for the new CSV file
data.to_csv(new_file_path, index=False)

print("Columns removed and new file saved successfully.")


Columns removed and new file saved successfully.


In [13]:
import pandas as pd

# Read the CSV file containing symbol and RSI
file_path = '/content/new_file.csv'  # Specify the path to your CSV file
data = pd.read_csv(file_path)

# Sort the data based on the 'symbol' column in ascending order
sorted_data = data.sort_values(by='RSI')

# Save the sorted data to a new CSV file
new_file_path = 'sorted_file.csv'  # Specify the path for the new CSV file
sorted_data.to_csv(new_file_path, index=False)

print("Data arranged in ascending order based on symbols and saved to a new file.")


Data arranged in ascending order based on symbols and saved to a new file.


In [14]:
import pandas as pd

# Load the CSV file into a DataFrame
file_path = "/content/new_file.csv"  # Update with your file path
df = pd.read_csv(file_path)

# Remove entries with RSI value greater than 70 and RSI value equals 0
filtered_df = df[(df['RSI'] <= 70) & (df['RSI'] != 0)]

# Optionally, save the filtered DataFrame to a new CSV file
output_file_path = "filtered_data.csv"  # Update with your desired output file path
filtered_df.to_csv(output_file_path, index=False)
print("Filtered data saved to", output_file_path)


Filtered data saved to filtered_data.csv


In [15]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import pytz

# Set the timezone to India (IST)
india_tz = pytz.timezone('Asia/Kolkata')

# Calculate the date range in the Indian timezone
end_date = datetime.now(india_tz)
start_date = end_date - timedelta(days=365)

# Read symbols from the file
symbols_file = "/content/filtered_data.csv"  # Update with your file path
symbols_df = pd.read_csv(symbols_file)

# Initialize lists to store conditions
symbols = []
close_above_ema_list = []
macd_above_signal_list = []
histogram_above_zero_list = []

# Iterate over symbols
for symbol in symbols_df['Symbol']:
    try:
        # Download data for the specified symbol and date range
        data = yf.download(symbol, start=start_date, end=end_date)

        # Calculate MACD
        exp12 = data['Close'].ewm(span=12, adjust=False).mean()
        exp26 = data['Close'].ewm(span=26, adjust=False).mean()
        macd = exp12 - exp26
        signal = macd.ewm(span=9, adjust=False).mean()
        macd_histogram = macd - signal

        # Calculate 200-day EMA
        ema_200 = data['Close'].ewm(span=200, min_periods=0, adjust=False).mean()

        # Get the last day's data
        last_day_data = data.iloc[-1]

        # Check conditions for the last day's data
        close_above_ema = last_day_data['Close'] > ema_200.iloc[-1]
        macd_above_signal = macd.iloc[-1] > signal.iloc[-1]
        histogram_above_zero = macd_histogram.iloc[-1] > 0

        # Append symbol and conditions to the lists
        symbols.append(symbol)
        close_above_ema_list.append(close_above_ema)
        macd_above_signal_list.append(macd_above_signal)
        histogram_above_zero_list.append(histogram_above_zero)

    except Exception as e:
        print(f"Error occurred for symbol {symbol}: {e}")

# Create a DataFrame for the conditions
conditions_df = pd.DataFrame({
    'Symbol': symbols,
    'Close above EMA': close_above_ema_list,
    'MACD above Signal': macd_above_signal_list,
    'Histogram above Zero': histogram_above_zero_list
})

# Save the DataFrame to a CSV file
output_file = "conditions_output.csv"  # Update with your desired output file path
conditions_df.to_csv(output_file, index=False)

print("Results have been saved to", output_file)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Results have been saved to conditions_output.csv


In [16]:
import pandas as pd

# Read the file into a DataFrame
file_path = "/content/conditions_output.csv"  # Update with your file path
df = pd.read_csv(file_path)

# Filter rows with True values in all three columns
filtered_df = df[(df['Close above EMA'] == True) &
                 (df['MACD above Signal'] == True) &
                 (df['Histogram above Zero'] == True)]

# Save the filtered DataFrame to a new CSV file
filtered_file_path = "filtered_output.csv"  # Update with your desired output file path
filtered_df.to_csv(filtered_file_path, index=False)

print("Filtered data has been saved to", filtered_file_path)


Filtered data has been saved to filtered_output.csv


In [17]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

def calculate_bollinger_bands(data, window=20, num_std=2):
    # Calculate rolling mean and standard deviation
    rolling_mean = data['Close'].rolling(window=window).mean()
    rolling_std = data['Close'].rolling(window=window).std()

    # Calculate upper and lower Bollinger Bands
    upper_band = rolling_mean + (rolling_std * num_std)
    lower_band = rolling_mean - (rolling_std * num_std)

    return upper_band, lower_band

def calculate_rsi(data, window=14):
    delta = data['Close'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

    RS = gain / loss
    RSI = 100 - (100 / (1 + RS))
    return RSI

# Get current date and time
current_time = datetime.now()

# Calculate start and end dates
start_date = current_time - timedelta(days=365)
end_date = current_time

# Read symbols from CSV
symbols_df = pd.read_csv('/content/filtered_output.csv')

results = []

for symbol in symbols_df['Symbol']:
    # Fetch stock data
    stock_data = yf.download(symbol, start=start_date, end=end_date)

    # Calculate Bollinger Bands
    upper_band, lower_band = calculate_bollinger_bands(stock_data)

    # Calculate RSI
    rsi = calculate_rsi(stock_data)

    # Get the latest data points
    latest_date = stock_data.index[-1]
    latest_close_price = stock_data['Close'].iloc[-1]
    latest_upper_band = upper_band.iloc[-1]
    latest_lower_band = lower_band.iloc[-1]
    latest_rsi = rsi.iloc[-1]

    # Check conditions
    close_above_upper_bb = latest_close_price > latest_upper_band
    close_below_lower_bb = latest_close_price < latest_lower_band
    rsi_near_30 = latest_rsi < 30
    rsi_near_70 = latest_rsi > 70

    results.append([symbol, close_above_upper_bb, close_below_lower_bb, rsi_near_30, rsi_near_70])

# Create DataFrame for results
results_df = pd.DataFrame(results, columns=['Symbol', 'Close above Upper BB', 'Close below Lower BB', 'RSI near 30', 'RSI near 70'])

# Save results to CSV
results_df.to_csv('analysis_results.csv', index=False)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******